## Домашняя работа по теме "Dataset, Dataloader, BatchNorm, Dropout, Оптимизация"

__Задание:__

1. Создать Dataset для загрузки данных (sklearn.datasets.fetch_california_housing)
2. Обернуть его в Dataloader
3. Написать архитектуру сети, которая предсказывает стоимость недвижимости. Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
4. Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

In [1]:
from sklearn.datasets import fetch_california_housing
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [2]:
california_housing = fetch_california_housing(as_frame=True)

Посмотрим на данные

In [3]:
california_housing.frame.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [4]:
california_housing.frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MedInc       20640 non-null  float64
 1   HouseAge     20640 non-null  float64
 2   AveRooms     20640 non-null  float64
 3   AveBedrms    20640 non-null  float64
 4   Population   20640 non-null  float64
 5   AveOccup     20640 non-null  float64
 6   Latitude     20640 non-null  float64
 7   Longitude    20640 non-null  float64
 8   MedHouseVal  20640 non-null  float64
dtypes: float64(9)
memory usage: 1.4 MB


In [5]:
X, y = fetch_california_housing(return_X_y=True, as_frame=True)

In [6]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13, test_size=0.25)

In [7]:
# Обернем датасет в Dataloader
X_train = torch.tensor(X_train.values)
y_train = torch.tensor(y_train.values).reshape(-1, 1)
X_test = torch.tensor(X_test.values)
y_test = torch.tensor(y_test.values).reshape(-1, 1)

train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=512,
                                          shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=512,
                                          shuffle=False)
                            

### Построим модель

У нас имеется 8 пизнаков и 1 таргет (цена). Следовательно, построим регрессионную модель. Также бужем использовать метод dropout

In [8]:
class Perceptron(nn.Module):
    def __init__(self, input_dim, output_dim, activation="relu"):
        super().__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.activation = activation

    def forward(self, x):
        x = self.fc(x)
        if self.activation == "relu":
            return F.relu(x)
        if self.activation == "sigmoid":
            return F.sigmoid(x)
        raise RuntimeError


class FeedForward(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, hidden_dim3):
        super().__init__()
        self.fc1 = Perceptron(input_dim, hidden_dim1, 'relu')
        #self.bn = nn.BatchNorm1d(hidden_dim)
        self.dp = nn.Dropout(0.10)
        self.fc2 = Perceptron(hidden_dim1, hidden_dim2, 'relu')
        self.fc3 = Perceptron(hidden_dim2, hidden_dim3, 'relu')
        self.fc4 = Perceptron(hidden_dim3, 1, "relu")

    def forward(self, x):
        x = self.fc1(x)
        x = self.dp(x)
        #x = self.bn(x)
        x = self.fc2(x)
        #x = self.dp(x)
        x = self.fc3(x)
        #x = self.dp(x)
        x = self.fc4(x)
        return x

In [9]:
net = FeedForward(8, 24, 12, 6)

Функция потерь и оптимайзер

In [10]:
criterion = nn.MSELoss(reduction='sum')  # mean square error
#optimizer = optim.Adam(net.parameters(), lr=0.00001)
#optimizer =  torch.optim.RMSprop(net.parameters(), lr=0.0001)

In [11]:
def train(num_epochs=200, optimizer=torch.optim.RMSprop(net.parameters(), lr=0.0001)):
    for epoch in range(num_epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(train_loader):
            batch_inputs, batch_labels = data[0], data[1]
            inputs, labels = batch_inputs[i].float(), batch_labels[i].float()
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward pass
            outputs = net(inputs)
            # defining loss
            loss = criterion(outputs, labels)
            # computing gradients
            loss.backward()
            # accumulating running loss
            running_loss += loss.item()
            # updated weights based on computed gradients
            optimizer.step()
            if i % 5000 == 0:
                net.eval()
                print('Epoch [%d]/[%d] running accumulative loss across all batches: %.3f' %
                    (epoch + 1, num_epochs, running_loss))
                running_loss = 0.0
                running_loss_test = 0.0
                for num, data2 in enumerate(test_loader):
                    batch_test_inputs, batch_test_labels = data2[0], data2[1]
                    test_inputs, test_labels = batch_test_inputs[num].float(), batch_test_labels[num].float()
                    test_outputs = net(test_inputs)
                    test_lost = criterion(test_outputs, test_labels)
                    running_loss_test += test_lost.item()
                print(f'Test err: {running_loss_test}')
                net.train()

    print('Training is finishing!')
    return running_loss_test

In [12]:
RMSprop_res = train()

Epoch [1]/[200] running accumulative loss across all batches: 7.182
Test err: 41.310118943452835
Epoch [2]/[200] running accumulative loss across all batches: 7.182
Test err: 41.310118943452835
Epoch [3]/[200] running accumulative loss across all batches: 7.182
Test err: 41.310118943452835
Epoch [4]/[200] running accumulative loss across all batches: 7.182
Test err: 41.310118943452835
Epoch [5]/[200] running accumulative loss across all batches: 7.182
Test err: 41.310118943452835
Epoch [6]/[200] running accumulative loss across all batches: 7.182
Test err: 41.310118943452835
Epoch [7]/[200] running accumulative loss across all batches: 7.182
Test err: 41.310118943452835
Epoch [8]/[200] running accumulative loss across all batches: 3.743
Test err: 41.310118943452835
Epoch [9]/[200] running accumulative loss across all batches: 7.182
Test err: 41.310118943452835
Epoch [10]/[200] running accumulative loss across all batches: 7.182
Test err: 41.310118943452835
Epoch [11]/[200] running accu

Изменим оптимайзер на Adam

In [13]:
net = FeedForward(8, 24, 12, 6)
Adam_res = train(optimizer=torch.optim.Adam(net.parameters(), lr=0.0001))

Epoch [1]/[200] running accumulative loss across all batches: 7.182
Test err: 41.310118943452835
Epoch [2]/[200] running accumulative loss across all batches: 7.182
Test err: 41.310118943452835
Epoch [3]/[200] running accumulative loss across all batches: 7.182
Test err: 41.310118943452835
Epoch [4]/[200] running accumulative loss across all batches: 7.182
Test err: 41.310118943452835
Epoch [5]/[200] running accumulative loss across all batches: 7.182
Test err: 41.310118943452835
Epoch [6]/[200] running accumulative loss across all batches: 7.182
Test err: 41.310118943452835
Epoch [7]/[200] running accumulative loss across all batches: 7.182
Test err: 41.310118943452835
Epoch [8]/[200] running accumulative loss across all batches: 7.182
Test err: 41.310118943452835
Epoch [9]/[200] running accumulative loss across all batches: 7.182
Test err: 41.310118943452835
Epoch [10]/[200] running accumulative loss across all batches: 7.182
Test err: 41.310118943452835
Epoch [11]/[200] running accu

Изменим оптимайзер на SGD

In [14]:
net = FeedForward(8, 24, 12, 6)
SGD_res = train(optimizer=torch.optim.SGD(net.parameters(), lr=0.0001))

Epoch [1]/[200] running accumulative loss across all batches: 7.182
Test err: 39.97309949994087
Epoch [2]/[200] running accumulative loss across all batches: 7.182
Test err: 28.650760028511286
Epoch [3]/[200] running accumulative loss across all batches: 5.242
Test err: 27.83342058956623
Epoch [4]/[200] running accumulative loss across all batches: 7.182
Test err: 27.589501881971955
Epoch [5]/[200] running accumulative loss across all batches: 5.157
Test err: 27.279007574543357
Epoch [6]/[200] running accumulative loss across all batches: 5.111
Test err: 26.97603945247829
Epoch [7]/[200] running accumulative loss across all batches: 5.064
Test err: 26.670686283148825
Epoch [8]/[200] running accumulative loss across all batches: 5.021
Test err: 26.392901240848005
Epoch [9]/[200] running accumulative loss across all batches: 4.980
Test err: 26.125078715384007
Epoch [10]/[200] running accumulative loss across all batches: 4.938
Test err: 25.854215297847986
Epoch [11]/[200] running accumul

In [15]:
print(f'Результаты модели с различными оптимайзерами (Ошибка MSE на тесте): \n'
      f'Adam: {Adam_res} \n'
      f'RMSprop: {RMSprop_res} \n'
      f'SGD: {SGD_res}')

Результаты модели с различными оптимайзерами (Ошибка MSE на тесте): 
Adam: 41.310118943452835 
RMSprop: 10.509002172388136 
SGD: 7.256959788966924


Лучше всего себя показал оптимайзер SGD